In [109]:
import numpy as np
import pandas as pd

s = pd.Series({'Google':[.2,.3,.4,-.2], 'Apple': [.3,.1,.3,.4],'Ebay':[.1,.7,-.1,-.2]})
x=s.values

# x = np.array([[0, 2 , 3], [1, 1, 6], [2, 0, 9]]).T

# np.cov(x)
# s = pd.Series({'a':[1,2], 'b':[2,3], 'c':[3,5]})
# u=np.array(s.values)
# u.astype(int)
# print(list(s.values))
returns=np.array(list(s.values))
cov_matrix=np.cov(np.array(list(s.values)))
print(cov)

[[ 0.02   0.01  -0.01 ]
 [ 0.01   0.005 -0.005]
 [-0.01  -0.005  0.005]]


In [110]:
# ticker= pd.Series({'Google':[1,2,3], 'Apple': [3,6,9], 'AMZN':[8,10,12]})
# cov_matrix=np.cov(np.array(list(ticker.values)))

In [111]:
diag=cov_matrix.diagonal()
diag_matrix=np.diag(np.diag(cov_matrix)) # gives n*n diagonal matrix
diag_matrix

array([[ 0.01583333,  0.        ,  0.        ],
       [ 0.        ,  0.1625    ,  0.        ],
       [ 0.        ,  0.        ,  0.06916667]])

In [112]:
from arch import arch_model
garch11 = arch_model(diag, p=1, q=1)
res = garch11.fit(update_freq=10)
print(res.summary())

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -4.19895560621
            Iterations: 6
            Function evaluations: 41
            Gradient evaluations: 6
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                      y   R-squared:                      -0.001
Mean Model:             Constant Mean   Adj. R-squared:                 -0.001
Vol Model:                      GARCH   Log-Likelihood:                4.19896
Distribution:                  Normal   AIC:                         -0.397911
Method:            Maximum Likelihood   BIC:                          -4.00346
                                        No. Observations:                    3
Date:                Thu, Feb 09 2017   Df Residuals:                       -1
Time:                        18:30:12   Df Model:                            4
                                  Mean Model                                  
     

In [113]:
alpha=res.params[2]
beta=res.params[3]
# std_err=np.random.normal(0, 1, 3) #epselon is the returns 

In [114]:
# esubt=np.linalg.inv(diag_matrix)
# ret=[.1,.2,.13] # just to see how it works
esubt=np.matmul(np.linalg.inv(diag_matrix),returns)
# esubt=np.matmul(np.linalg.inv(diag_matrix),returns)
esubt

array([[ 18.94736842,   6.31578947,  18.94736842,  25.26315789],
       [  0.61538462,   4.30769231,  -0.61538462,  -1.23076923],
       [  2.89156627,   4.3373494 ,   5.78313253,  -2.89156627]])

In [115]:
#not sure
qdash=np.cov(esubt)
q_start=np.ones((3,3))
qdash
# q_start

array([[ 63.15789474, -19.10931174, -19.02346227],
       [-19.10931174,   6.15384615,   4.22613531],
       [-19.02346227,   4.22613531,  14.45783133]])

In [116]:
q=((1-alpha-beta)*qdash) + (alpha*np.matmul(esubt,esubt.T)) + ((beta)*q_start)
q

array([[ 9.10497541, -1.62212019, -1.61092597],
       [-1.62212019,  1.67202718,  1.42066654],
       [-1.61092597,  1.42066654,  2.7548115 ]])

In [117]:
# r=np.matmul(np.linalg.inv(qdash),q)
qstar=np.diag(np.diag(cov_matrix))
qstar=np.sqrt(qstar)
qstar

array([[ 0.12583057,  0.        ,  0.        ],
       [ 0.        ,  0.40311289,  0.        ],
       [ 0.        ,  0.        ,  0.26299556]])

In [118]:
r=np.matmul(np.linalg.inv(qstar),q,np.linalg.inv(qstar))
r

array([[ 72.35900721, -12.89130408, -12.8023414 ],
       [ -4.02398495,   4.14778896,   3.52423995],
       [ -6.1252971 ,   5.40186504,  10.47474513]])

In [119]:
diag_matrix=np.diag(np.diag(cov_matrix))
print(diag_matrix)
predi_cov=np.matmul(diag_matrix,r)
predi_cov=np.matmul(predi_cov,diag_matrix)
print(predi_cov)

[[ 0.01583333  0.          0.        ]
 [ 0.          0.1625      0.        ]
 [ 0.          0.          0.06916667]]
[[ 0.01814    -0.03316825 -0.01402034]
 [-0.01035338  0.10952755  0.03961099]
 [-0.00670805  0.06071471  0.05011147]]
